In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import numpy as np
import time
import re

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
aeropuertos = ["JFK","LAX","MIA"] 
urls = []
for x in aeropuertos:
    amx_url = f"https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_{x}_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0"
    urls.append(amx_url)
    print(amx_url)

https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_JFK_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_LAX_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0
https://aeromexico.com/es-mx/reserva/opciones?itinerary=MEX_MIA_2019-11-15&leg=1&travelers=A1_C0_I0_PH0_PC0


In [4]:
amx_data = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    try:
        browser.find_by_xpath("/html/body/div[2]/div/div/div[1]/div/div[1]/div/div/button").click()
    except:
        print("ok")
    amx_site = browser.html
    amx_soup = bs(amx_site, 'html.parser')
    options = amx_soup.find_all('div', class_="FlightOptionsListItem") #asegurate de que la ventana sea compacta
    print("appending")
    amx_data.append(options)

appending
ok
appending
ok
appending


In [5]:
precios = []
origenes = []
destinos = []
salidas = []
llegadas = []

for x in amx_data:
    for y in x:
        precio = y.find('div', class_='FlightOptionsFare-price')
        origen = y.find_all('div', class_='FlightOptionsTimeline-city')
        horario = y.find_all('div', class_='FlightOptionsTimeline-time')
        try:
            precios.append(precio.span.text)
            origenes.append(origen[0].text)
            destinos.append(origen[1].text)
            salidas.append(horario[0])
            llegadas.append(horario[1])
        except:
            continue

In [6]:
salidas2 = []

for x in salidas:
    try:
        salidas2.append(x.text)
    except:
        continue

In [7]:
llegadas2 = []

for x in llegadas:
    try:
        llegadas2.append(x.text)
    except:
        continue

In [8]:
df = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas2,
"Hora Llegada": llegadas2,
"Desde": precios 
})
df

,Origen,Destino,Hora Salida,Hora Llegada,Desde
0,MEX,JFK,07:15,13:00,$9259
1,MEX,JFK,09:20,15:20,$7768
2,MEX,JFK,15:00,20:54,"$11,322"
3,MEX,JFK,18:10,00:10,$8442
4,MEX,JFK,07:00,14:30,"$16,202"
5,MEX,JFK,06:30,14:30,"$16,202"
6,MEX,JFK,11:50,20:10,"$29,443"
7,MEX,JFK,06:01,14:30,"$16,202"
8,MEX,JFK,09:05,17:35,"$18,762"
9,MEX,JFK,13:20,21:51,"$29,443"


In [9]:
df.to_csv("aeromexico.csv")

In [10]:
aeropuertos = ["JFK","LAX","MIA"]
urls = []
for x in aeropuertos:
    amx_url = f"https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd={x}&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web"
    urls.append(amx_url)
    print(amx_url)

https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=JFK&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=LAX&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web
https://sales.avianca.com/B2C/InicioAmadeus.aspx?cco=MEX&Pais=MX&lan=ES&tv=false&fi=15NOV&FFL=false&ccd=MIA&fr=15NOV&Cabina=0&na=1&nn=0&ni=0&VInt=si&tt=4&ccorp=0&hvi=0&hvr=0&tarifa=0&SistemaOrigen=AH&FriendlyID=&FriendlyIDNegoF=&WS=&MPD=0&IvaMPD=0&descq=0&Device=web


In [11]:
avi_prices = []
avi_horarios = []

for x in urls:
    browser.visit(x)
    time.sleep(15) #aunque esperes mucho tiempo, dependes de la respuesta del servidor de la página
    avi_site = browser.html
    avi_soup = bs(avi_site, 'html.parser')
    avi_prices.append(avi_soup.find_all('div', class_='availability-list-fares'))
    avi_horarios.append(avi_soup.find_all('div', class_='flight-details availability-flight-details flight-details-without-button availability-flight-details-without-button row'))
    print("appending")

appending
appending
appending


In [12]:
precios = []

for x in avi_prices:
    for y in x:
        precios.append(y.find('div', class_='cell-reco-price').span.text)

In [13]:
salidas = []
llegadas = []
origenes = []
destinos = []

for x in avi_horarios:
    for y in x:
        salidas.append(y.find('time', class_='time-from').text)
        llegadas.append(y.find('time', class_='time-to').text)
        origenes.append(y.find('abbr', class_='citycode-from').text.replace("(","").replace(")",""))
        destinos.append(y.find('abbr', class_='citycode-to').text.replace("(","").replace(")",""))

In [14]:
df2 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas,
"Hora Llegada": llegadas,
"Desde": precios
})
df2

,Origen,Destino,Hora Salida,Hora Llegada,Desde
0,MEX,JFK,16:50,01:30,"8,176.00"
1,MEX,JFK,23:20,11:50,"36,725.00"
2,MEX,JFK,13:50,02:50,"25,636.00"
3,MEX,JFK,05:00,18:55,"32,069.00"
4,MEX,JFK,04:50,21:00,"7,496.00"
5,MEX,JFK,23:20,18:55,"36,725.00"
6,MEX,JFK,04:50,01:30,"7,634.00"
7,MEX,JFK,05:00,02:50,"27,679.00"
8,MEX,JFK,13:50,11:50,"27,679.00"
9,MEX,JFK,05:00,21:00,"29,292.00"


In [15]:
df2.to_csv("avianca.csv")

In [16]:
aeropuertos = ["JFK","LAX","MIA"]
urls = []
for x in aeropuertos:
    ua_url = f"https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t={x}&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1"
    urls.append(ua_url)
    print(ua_url)

https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t=JFK&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1
https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t=LAX&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1
https://www.united.com/ual/en/MX/flight-search/book-a-flight/results/?f=MEX&t=MIA&d=2019-11-15&tt=1&sc=7&px=1&taxng=1&newHP=True&idx=1


In [17]:
ua_data = []

for x in urls:
    browser.visit(x)
    time.sleep(5)
    browser.visit(x)
    time.sleep(15)
    united_site = browser.html
    united_soup = bs(united_site, 'html.parser')
    print("appending")
    ua_data.append(united_soup.find_all('li', class_='flight-block flight-block-fares use-roundtrippricing flight-block-revised'))

appending
appending
appending


In [18]:
salidas = []
llegadas = []
origenes = []
destinos = []
precios = []

for x in ua_data:
    for y in x:
        xx = y.find('div', class_='flight-time flight-time-depart').text
        xy = y.find('div', class_='flight-time flight-time-arrive').text
        xz = y.find_all('span')
        xw = (y.find('div', class_='price-point price-point-revised use-roundtrippricing').text)
        salidas.append(re.search(r'\d{1,2}(:\d{1,2})', xx).group())
        llegadas.append(re.search(r'\d{1,2}(:\d{1,2})', xy).group())
        origenes.append(xz[1].text)
        destinos.append(xz[3].text)
        precios.append(xw.replace("\n", "").replace("US", "").replace("$",""))

In [19]:
df3 = pd.DataFrame({
"Origen": origenes,
"Destino": destinos,
"Hora Salida": salidas,
"Hora Llegada": llegadas,
"Desde": precios
})
df3["Origen"] = df3["Origen"].str[-4:]
df3["Destino"] = df3["Destino"].str[-4:]
df3["Origen"] = df3["Origen"].str[:3]
df3["Destino"] = df3["Destino"].str[:3]
df3

,Origen,Destino,Hora Salida,Hora Llegada,Desde
0,MEX,EWR,9:05,14:50,396
1,MEX,EWR,13:55,19:40,396
2,MEX,EWR,5:40,14:50,402
3,MEX,LGA,5:40,16:47,402
4,MEX,EWR,5:40,19:10,402
5,MEX,EWR,6:05,19:40,"1,265"
6,MEX,LGA,7:50,16:47,402
7,MEX,EWR,7:50,19:10,402
8,MEX,EWR,7:50,20:51,402
9,MEX,LGA,7:50,21:59,402


In [20]:
df3.to_csv("united.csv")